In [1]:
import torch
import pickle
from sklearn.preprocessing import LabelEncoder
from sctokenizer import CTokenizer

from models import StackLSTM

In [2]:
from collections import defaultdict

In [8]:
from tqdm import tqdm

In [3]:
vocab_path = "vocab.pkl"

In [4]:
with open(vocab_path, "rb") as input_file:
    vocab = pickle.load(input_file)
le = LabelEncoder()
le.fit(["<SOC>", "<EOC>"])
le.fit(list(vocab))


LabelEncoder()

In [5]:
def process_and_encode(text):
    tokenizer = CTokenizer()
    all_tokens = tokenizer.tokenize(text)
    filtered = [token for token in all_tokens if token.token_value in vocab]
    values = [token.token_value for token in filtered]
    lines = [token.line for token in filtered]
    code = torch.tensor(le.transform(values))
    return code, lines

In [6]:
HIDDEN_SIZE_CONTROLLER = 8
EMBED_DIM = 164
HIDDEN_SIZE_STACK = 8

In [7]:
model = StackLSTM(embedding_size=len(vocab),
                  embedding_dim=EMBED_DIM,
                  hidden_size_controller=HIDDEN_SIZE_CONTROLLER,
                  hidden_size_stack=HIDDEN_SIZE_STACK,
                  batch_size=1,
                  label_encoder=le)
model.load_state_dict(torch.load("model_7"))
model.eval()

StackLSTM(
  (embedding): Embedding(10001, 164)
  (controller): LSTMCell(172, 8)
  (output_linear): Linear(in_features=8, out_features=10001, bias=True)
  (softmax): Softmax(dim=None)
  (push_fc): Linear(in_features=8, out_features=1, bias=True)
  (pop_fc): Linear(in_features=8, out_features=1, bias=True)
  (values_fc): Linear(in_features=8, out_features=8, bias=True)
  (classifier): Linear(in_features=10001, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [9]:
from datasets import load_from_disk

In [27]:
dataset = load_from_disk("../../../codet5-linevul/reveal/")
dataset = dataset.filter(lambda r: len(r["processed_func"]) < 1_000).shuffle()

Loading cached processed dataset at /home/imger/repos/codet5-linevul/reveal/cache-78fd635ed2c5ef06.arrow


In [28]:
preds = []
trues = []
for sample in tqdm(dataset):
    data, lines = process_and_encode(sample["processed_func"])
    pred = model(data.unsqueeze(1))
    preds.append(pred)
    trues.append(sample["label"])

  7%|▋         | 1098/15445 [44:41<9:43:51,  2.44s/it]   


KeyboardInterrupt: 

In [20]:
from sklearn.metrics import matthews_corrcoef, accuracy_score, f1_score, balanced_accuracy_score

def compute_metrics(pred, true):
    predicted = (torch.as_tensor(pred) > 0.5).long().tolist()
    return {
        "MCC": matthews_corrcoef(true, predicted),
        "F1": f1_score(true, predicted, average='macro'),
        "Acc": accuracy_score(true, predicted),
        "BAcc": balanced_accuracy_score(true, predicted),
    }

In [29]:
compute_metrics(preds, trues)

{'MCC': 0.06857720500493818,
 'F1': 0.4700239003854101,
 'Acc': 0.6985428051001822,
 'BAcc': 0.5729619933194908}